In [ ]:
# pip install the required packages if running in Colab
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
# Standard imports, plotting option and constant definition
from datetime import datetime, timedelta
from typing import List, Union
import pandas as pd
import plotly.express as px
import numpy as np

from summer import CompartmentalModel
from summer import Stratification

pd.options.plotting.backend = "plotly"

COVID_BASE_DATE = datetime(2019, 12, 31)
region = "Malaysia"

In [ ]:
# Get a function to access the Malaysia data if running in Colab
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/get_mys_data.py

import get_mys_data

In [ ]:
# ... and use it to get the actual data
df = get_mys_data.fetch_mys_data()
initial_population = get_mys_data.get_initial_population(region)
observations = get_mys_data.get_target_observations(df, region, "cases")

In [ ]:
# Define the model running period and convert to a numeric representation
start_date = datetime(2021, 1, 30)
end_date = start_date + timedelta(days=300)
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

In [ ]:
# Define a target set of observations to compare against our modelled outputs later
notifications_target = observations[start_date: end_date]["cases_new"]

# Model
## Define a model

In [ ]:
def build_unstratified_model(parameters: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        parameters: Flow parameters
    Returns:
        A compartmental model currently without stratification applied
    """

    model = CompartmentalModel(
        times=(parameters["start_time"], parameters["end_time"]),
        compartments=["S", "E", "I", "R"],
        infectious_compartments=["I"],
        ref_date=COVID_BASE_DATE
    )

    infectious_seed = parameters["infectious_seed"]

    model.set_initial_population(distribution={"S": initial_population - infectious_seed, "E": 0, "I": infectious_seed})
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="E"
    )
    # Expose people transition to infected
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    # Add an infection-specific death flow to the I compartment
    model.add_death_flow(name="infection_death", death_rate=parameters["death_rate"], source="I")

    # We will also request an output for the 'progression' flow
    model.request_output_for_flow("progressions", "progression")

    return model

# strain stratification

In [ ]:
from summer import StrainStratification
def get_strain_stratification(
    compartments_to_stratify: List[str], 
    strata: List[str],
    voc_params: dict
) -> Stratification:
    """
    Create a summer stratification object that stratifies compartments into
    strata, which are intended to represent infectious disease strains.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify
        strata: The strata to be implemented in the strain stratification
        voc_params: A dictionary which speicifies the infectiousness and severity of strains
    Returns:
        A summer stratification object to represent strain stratification (not yet applied)
    """
    
    strat = StrainStratification(name="strain", strata=strata, compartments=compartments_to_stratify)

    # At the start of the simulation, 20% of infected people have wild strain.
    strat.set_population_split({'variant': 0.8, 'wild': 0.2})

    # adjusting the severity of wild strain and variant
    strat.set_flow_adjustments("infection_death", {
        "variant": voc_params["variant_ifr"],
        "wild": voc_params["wild_ifr"],
    })


    # Adjusting the infectiousness of Wild strain and variant.
    strat.set_flow_adjustments("infection", {
        "variant": voc_params["variant_infectiousness"],
        "wild": voc_params["wild_infectiousness"],
    })
    
    return strat

### Combine the two processes together

In [ ]:
def build_stratified_model(
    parameters: dict,
    strata: List[str],
    compartments_to_stratify:List[str],
    voc_params: dict
) -> CompartmentalModel:
    """
    Get the model object with the strain stratification applied to it.
    
    Arguments:
        parameters: A dictionary containing the parameter values to use
        strata: The strata to be implemented in the strain stratification
        compartments_to_stratify: A list of compartments to which the stratification is applied
        voc_params: A dictionary which speicifies the infectiousness and severity of strains
    Returns:
        The model object        
    """
    
    # Get an unstratified model object
    model = build_unstratified_model(parameters)

    # Get and apply the stratification
    strain_strat = get_strain_stratification(compartments_to_stratify, strata, voc_params)
    model.stratify_with(strain_strat)

    return model

### Actually run the models

In [ ]:
# Build and run the stratified model with some arbitrary parameters
simple_strain_params = {
    "contact_rate": 0.5,
    "progression_rate": 0.2,
    "recovery_rate": 0.2,
    "death_rate": 0.,
    "reporting_fraction": 0.02,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 100.
}

voc_params = {
    "variant_ifr": 1.2,
    "wild_ifr": None,
    "variant_infectiousness": 2.,
    "wild_infectiousness": None,
}

strata = ["variant", "wild"]
compartments_to_stratify = ["E", "I", "R"]

simple_strain_strat_model = build_stratified_model(simple_strain_params, strata, compartments_to_stratify, voc_params)

In [ ]:
simple_strain_strat_model.run()

progressions = simple_strain_strat_model.get_derived_outputs_df()["progressions"]
simple_cases_modelled = progressions * simple_strain_params["reporting_fraction"]

pd.DataFrame(
    {
        "observed": notifications_target,
        "modelled": simple_cases_modelled,
    }
).plot()

## Manual calibration to notifications

In [ ]:
def voc_adjust_variant_ifr(time, computed_values):
    if time > 500:
        variant_ifr = 2.5  
    else:
        variant_ifr = 1.5   
    return variant_ifr    

In [ ]:
def voc_adjust_variant_infectiousness(time, computed_values):
    if time > 500:
        variant_infectiousness = 1.5  
    else:
        variant_infectiousness = 1.15   
    return variant_infectiousness   

In [ ]:
custom_voc_params = {
    "variant_ifr": voc_adjust_variant_ifr,
    "wild_ifr": 1,
    "variant_infectiousness": voc_adjust_variant_infectiousness,
    "wild_infectiousness": 1,
}


custom_params = {
        "contact_rate": 0.225,
        "progression_rate": 0.2,
        "recovery_rate": 0.2,
        "death_rate": 0.01,
        "reporting_fraction": 0.095,
        "start_time": start_date_int,
        "end_time": end_date_int,
        "infectious_seed": 100.
    }

In [ ]:
custom_strain_strat_model = build_stratified_model(custom_params, strata, compartments_to_stratify, custom_voc_params)
custom_strain_strat_model.run()

progressions = custom_strain_strat_model.get_derived_outputs_df()["progressions"]
custom_cases_modelled = progressions * custom_params["reporting_fraction"]

pd.DataFrame(
    {
        "observed": notifications_target,
        "modelled": custom_cases_modelled,
    }
).plot()